In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import pickle

In [9]:
with open('global_travel_tensor.pkl', 'rb') as f:
    travel_tensor = pickle.load(f) 

with open('global_serial_graphs.pkl', 'rb') as f:
    serial_graphs = pickle.load(f) 

In [6]:

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    # to install networkx 2.0 compatible version of python-louvain use:
    # pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
    from community import community_louvain

    g = nx.karate_club_graph()
    partition = community_louvain.best_partition(g)
    pos = community_layout(g, partition)

    nx.draw(g, pos, node_color=list(partition.values())); plt.show()
    return

In [52]:
A=travel_tensor[100][:,0:226]
G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())

In [53]:
G
# nx.algorithms.community.modularity(G, [{0, 1, 2}, {3, 4, 5}])
greedy_modularity_communities(G)

IndexError: list index out of range

In [42]:
from networkx.algorithms import community
communities_generator = community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)

In [43]:
sorted(map(sorted, next_level_communities))

[[0,
  2,
  3,
  4,
  9,
  10,
  11,
  12,
  14,
  16,
  17,
  18,
  20,
  21,
  23,
  24,
  25,
  26,
  27,
  29,
  32,
  33,
  36,
  37,
  38,
  40,
  41,
  43,
  44,
  45,
  46,
  49,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  62,
  63,
  64,
  65,
  66,
  67,
  70,
  71,
  73,
  74,
  77,
  79,
  80,
  89,
  91,
  99,
  100,
  105,
  110,
  111,
  116,
  121,
  123,
  127,
  130,
  131,
  158,
  176,
  196],
 [1],
 [5, 6, 31],
 [7, 22, 28, 30, 42, 48, 96],
 [8],
 [13, 87],
 [15],
 [19],
 [34],
 [35],
 [39],
 [47],
 [50],
 [51],
 [52],
 [61],
 [68],
 [69],
 [72],
 [75],
 [76],
 [78],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [88],
 [90],
 [92],
 [93],
 [94],
 [95],
 [97],
 [98],
 [101],
 [102],
 [103],
 [104],
 [106],
 [107],
 [108],
 [109],
 [112],
 [113],
 [114],
 [115],
 [117],
 [118],
 [119],
 [120],
 [122],
 [124],
 [125],
 [126],
 [128],
 [129],
 [132],
 [133],
 [134],
 [135],
 [136],
 [137],
 [138],
 [139],
 [140],
 [141],
 [142],
 [143],
 [144],
 [145],
 [146],
 [147

In [38]:
from community import community_louvain
# test()
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)

nx.draw(G, pos, node_color=list(partition.values())); plt.show()

TypeError: Bad graph type, use only non directed graph

In [23]:
B = nx.directed_modularity_matrix(G)
B

NetworkXNotImplemented: not implemented for undirected type